# ML4HPC: Ensemble of Forecasters

### Team Members:
- Luca Venerando Greco
- Bice Marzagora
- Elia Vaglietti


### Importing Libraries

In this notebook, we start by importing several essential libraries that are used throughout the workflow:

- `os`: Provides functions for interacting with the operating system, such as creating directories and handling file paths.
- `sys`: Provides access to some variables used or maintained by the Python interpreter and to functions that interact strongly with the interpreter.
- `matplotlib.pyplot`: A plotting library used for creating static, animated, and interactive visualizations in Python.
- `numpy`: A fundamental package for scientific computing with Python, used for working with arrays and matrices.
- `time`: Provides various time-related functions.
- `tqdm`: A library for creating progress bars and progress meters.

These libraries are crucial for tasks such as data manipulation, visualization, and managing the execution of jobs in a high-performance computing environment.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
import jax
import jax.numpy as jnp
from jax import grad
import time
import os
import tqdm

2024-12-02 15:06:49.134843: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 15:06:49.140195: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 15:06:49.152604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733148409.170811  624293 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733148409.176549  624293 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 15:06:49.197588: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

### Directory Setup and Job Configuration

We now set up the necessary directories and define the job configurations. Specifically, we create folders for storing charts, data, and logs if they do not already exist. We also define the number of forecasters and nodes for different scaling tests.

If no new data is needed, set the `GENERATE_DATA` variable to `False` to skip the data generation step.


In [2]:
current_dir = os.getcwd()

charts_folder = "charts"
data_folder = "data"
logs_folder = "logs"

if not os.path.exists(data_folder):
    os.makedirs(data_folder)

if not os.path.exists(logs_folder):
    os.makedirs(logs_folder)

if not os.path.exists(charts_folder):
    os.makedirs(charts_folder)

ten_nodes = 10
strong_scaling_nodes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # Number of nodes to test
weak_scaling_nodes   = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # Number of nodes to test

n_runs = 30

GENERATE_DATA = True

### Job Submission Function

We define a function `submit_job` that handles the submission of jobs to the scheduler. This function takes the number of nodes, the number of forecasters, and a job name as input parameters. It creates the necessary directories for storing data and logs, reads a template launch script, formats it with the provided parameters, writes the formatted script to a file, and submits the job using the `sbatch` command.

In [3]:
def submit_job(launch_file, num_nodes, job_name, num_epochs):
    num_tasks_per_node = 128

    if num_tasks_per_node > 128:
        print("The number of tasks per node should be less than or equal to 128")
        exit(1)

    if not os.path.exists(f"{data_folder}/{job_name}"):
        os.makedirs(f"{data_folder}/{job_name}")

    if not os.path.exists(f"{logs_folder}/{job_name}"):
        os.makedirs(f"{logs_folder}/{job_name}")

    with open(launch_file, 'r') as file:
        launch_script = file.read()

    launch_script = launch_script.format(
        num_nodes=num_nodes,
        num_tasks_per_node=num_tasks_per_node,
        current_dir=current_dir,
        world_size=num_nodes*num_tasks_per_node,
        num_epochs=num_epochs,
        data_folder=f"{data_folder}/{job_name}",
        logs_folder=f"{logs_folder}/{job_name}"
    )

    script_filename = f"{logs_folder}/{job_name}/{launch_file.split('/')[-1]}"
    with open(script_filename, "w") as script_file:
        script_file.write(launch_script)

    os.system(f"sbatch {script_filename}")

### Defining Test Functions

In the following sections, we define functions to run different scalability tests. These functions will help us automate the process of submitting jobs for one million forecasters, strong scaling, and weak scaling tests. Each function will generate a unique job name, submit the job using the `submit_job` function, and return the job names for tracking purposes.

In [4]:
def ten_nodes_test():
    job_names = []
    for i in range(n_runs):
        run_dir = f"{data_folder}/ten_nodes_test/run_{i}"
        if not os.path.exists(run_dir):
            os.makedirs(run_dir)
        
        job_name = f"/ten_nodes_test/run_{i}"

        submit_job(ten_nodes, job_name)

        job_names.append(job_name)

    return job_names

In [5]:
def strong_scaling():
    job_names = []
    for run in range(1):
        for num_nodes in strong_scaling_nodes:
            job_name = f"/strong_scaling/run_{run}/nodes_{num_nodes}"
            submit_job(num_nodes, job_name)
            job_names.append(job_name)
    return job_names

In [ ]:
# def weak_scaling():
#     job_names = []
#     for run in range(1):
#         for num_nodes in weak_scaling_nodes:
#             job_name = f"/weak_scaling/run_{run}/nodes_{num_nodes}_forecasters_{weak_scaling_forecasters*num_nodes}"
#             submit_job(num_nodes, job_name)
#             job_names.append(job_name)
#     return job_names

In [6]:
def baseline_test():
    job_names = []
    for i in range(n_runs):
        run_dir = f"{data_folder}/baseline_test/run_{i}"
        if not os.path.exists(run_dir):
            os.makedirs(run_dir)
        
        job_name = f"/baseline_test/run_{i}"

        submit_job("launch_baseline.sh", 1, job_name, 10)

        job_names.append(job_name)

    return job_names

### Waiting for jobs

Now we wait for all the jobs to complete, in the meantime the `tqdm` progress bar will be updated.

In [7]:
all_jobs_to_wait = []

if GENERATE_DATA:
    # all_jobs_to_wait.extend(baseline_test())
    # all_jobs_to_wait.extend(ten_nodes_test())
    # all_jobs_to_wait.extend(strong_scaling())
    # all_jobs_to_wait.extend(weak_scaling())

    print("Waiting for jobs to finish...")
    print(all_jobs_to_wait)

Submitted batch job 4580471
Submitted batch job 4580472
Submitted batch job 4580473
Submitted batch job 4580474
Submitted batch job 4580475
Submitted batch job 4580476
Submitted batch job 4580477
Submitted batch job 4580478
Submitted batch job 4580479
Submitted batch job 4580480
Submitted batch job 4580481
Submitted batch job 4580482
Submitted batch job 4580483
Submitted batch job 4580484
Submitted batch job 4580485
Submitted batch job 4580486
Submitted batch job 4580487
Submitted batch job 4580488
Submitted batch job 4580489
Submitted batch job 4580490
Submitted batch job 4580491
Submitted batch job 4580492
Submitted batch job 4580493
Submitted batch job 4580494
Submitted batch job 4580495
Submitted batch job 4580496
Submitted batch job 4580497
Submitted batch job 4580498
Submitted batch job 4580499
Submitted batch job 4580500
Waiting for jobs to finish...
['/baseline_test/run_0', '/baseline_test/run_1', '/baseline_test/run_2', '/baseline_test/run_3', '/baseline_test/run_4', '/baselin

In [ ]:
for job_name in tqdm(all_jobs_to_wait):
    while not os.path.exists(f"{data_folder}/{job_name}/timings.txt"):
        time.sleep(10)  # Poll every 10 seconds

### Timing Analysis

In this section, we analyze the execution times for the one million forecasters test. We read the timing data from the generated files, calculate the mean and standard deviation of the execution times, and create a dataframe to summarize the results.

The dataframe includes the following columns:
- **Run**: The run identifier.
- **Timing**: The total execution time for each run.
- **Aggregate sum CPU time**: The sum of CPU times across all ranks for each run.
- **Aggregate mean CPU time**: The mean CPU time across all ranks for each run.

We then print the dataframe and the calculated mean and standard deviation of the execution times.

In [ ]:
import pandas as pd

timings = []
cpu_times = []

for i in range(n_runs):
    timing_file = f"{data_folder}/ten_nodes_test/run_{i}/timings.txt"
    cpu_times_ranks = []
    with open(timing_file, "r") as f:
        line = f.readline().strip()
        timing = float(line.replace("Total execution time: ", ""))
        timings.append(timing)
        for line in f:
            cpu_times_ranks.append(float(line.strip().split(":")[1]))
    cpu_times.append(cpu_times_ranks)

mean_timing = np.mean(timings)
std_timing = np.std(timings)

# Create a dataframe
df_timings = pd.DataFrame({
    'Run': [f'run_{i}' for i in range(n_runs)],
    'Timing': timings,
    'Aggregate sum CPU time': [np.sum(cpu_times[i]) for i in range(n_runs)],
    'Aggregate mean CPU time': [np.mean(cpu_times[i]) for i in range(n_runs)]
})

print(df_timings)

# calculate the mean and std of the timings
mean_timing = np.mean(df_timings['Timing'])
std_timing = np.std(df_timings['Timing'])


print(f"Mean timing: {mean_timing}")
print(f"Std timing: {std_timing}")

### Strong Scalability Test

In this section, we analyze the execution times for the strong scalability test. We have already submitted jobs for different numbers of nodes and collected the execution times. The results are plotted on a logarithmic scale to better visualize the differences in execution times as the number of nodes increases.

The strong scalability test helps us understand how the execution time decreases as we increase the number of nodes while keeping the problem size constant. Ideally, the execution time should decrease proportionally with the increase in the number of nodes, indicating efficient parallelization and resource utilization.

In [ ]:
execution_times_strong_scaling = []

# Submit jobs for each test configuration
for num_nodes in strong_scaling_nodes:
    execution_time_file = f"{data_folder}/strong_scaling/run_0/nodes_{num_nodes}/timings.txt"

    with open(execution_time_file, "r") as f:
        line = f.readline().strip()
        execution_time = float(line.replace("Total execution time: ", ""))
    execution_times_strong_scaling.append(execution_time)
    print(f"Execution time for {num_nodes} nodes: {execution_time} seconds")


In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.yscale('log')
plt.plot(strong_scaling_nodes, execution_times_strong_scaling, label='Strong Scaling', marker='o')
plt.xlabel('Number of Nodes')
plt.ylabel('Execution Time (seconds)')
plt.title('Strong Scalability Test')
plt.grid(True)
plt.legend()
plt.savefig(f"{charts_folder}/scalability_plot.png") 
plt.show()

### Weak Scalability Test

In this section, we analyze the execution times for the weak scalability test. 

The weak scalability test helps us understand how the execution time changes as we increase the number of nodes while keeping the workload per node constant. Ideally, the execution time should remain constant with the increase in the number of nodes, indicating efficient parallelization and resource utilization.

In [ ]:
# execution_times_weak_scaling = []

# # Submit jobs for each test configuration
# for num_nodes in weak_scaling_nodes:
#     execution_time_file = f"{data_folder}/weak_scaling/run_0/nodes_{num_nodes}_forecasters_{weak_scaling_forecasters*num_nodes}/timings.txt"

#     with open(execution_time_file, "r") as f:
#         line = f.readline().strip()
#         execution_time = float(line.replace("Total execution time: ", ""))
#     execution_times_weak_scaling.append(execution_time)

In [ ]:

# # Plot the weak scalability results
# plt.figure(figsize=(10, 6))
# plt.plot(weak_scaling_nodes, execution_times_weak_scaling, label='Weak Scaling', marker='o')
# plt.xlabel("Number of Nodes")
# plt.ylabel("Execution Time (seconds)")
# plt.title("Weak Scalability Test")
# plt.grid(True)
# plt.savefig(f"{charts_folder}/weak_scalability.png")
# plt.show()

### Conclusion

In this notebook, we have successfully set up and executed a series of scalability tests for an ensemble of forecasters in a high-performance computing environment. We started by importing the necessary libraries and setting up the directory structure. We then defined functions to submit jobs for one million forecasters, strong scaling, and weak scaling tests.

We visualized the data generated from the one million forecasters test, analyzing the distribution of forecast, weights, and biases. The histograms confirmed that these variables are normally distributed, as expected.

The strong scalability test demonstrated how the execution time decreases with an increasing number of nodes, indicating efficient parallelization. The weak scalability test showed that the execution time remains relatively constant as the number of nodes increases, suggesting good resource utilization.

Overall, these tests provide valuable insights into the performance and scalability of our forecasting ensemble, helping us optimize and improve our high-performance computing workflows.